In [1]:
import pandas as pd
import random
from hold_em import deal_holdem
from seven_card_stud import deal, hand_types
from utils7card import * 
import matplotlib.pyplot as plt
%matplotlib inline

### Building off of [5_card_stud_tests.ipynb](5_card_stud_tests.ipynb), lets create distibutions of hand types for three poker games:
- 5-Card Stud
- 7-Card Stud
- Texas Hold 'em

### The question is to have a fair representation of each hand type, how many simulations should be run?
- First, lets find the distribution of the 9 handranks for every 5-card hand combination in a 52-card deck. 
- From the previous ipython notebook, we found out that there are *2,598,960* combinations. n_choose_k(52, 5)


In [2]:
hands = [h for h in itertools.combinations(deck, 5)] # for every combination
hand_ranks = [hand_rank(hand)[0] for hand in hands] # find the hand rank of each hand
len(hand_ranks) # sanity check

2598960

### Hand Values 
- Based on how poker.py was refactored, (changes here: [poker_refactored.py](poker_refactored.py))

The new rankings work out as such: 
- 9: "Straight flush", 
- 7: "4 of a kind",
- 6: "Full house",  
- 5: "Flush", 
- 4: "Straight",
- 3: "3 of a kind", 
- 2: "2 pair", 
- 1: "1 pair", 
- 0: "High card"

*Note that "straight flush" needed to be bumped to 9, leaving 8 without a partner in crime.*

In [3]:
hand_counts = [(hand_types[i], hand_ranks.count(i)) for i in range(10) if i!=8]
hand_counts

[('High card', 1302540),
 ('1 pair', 1098240),
 ('2 pair', 123552),
 ('3 of a kind', 54912),
 ('Straight', 10200),
 ('Flush', 5108),
 ('Full house', 3744),
 ('4 of a kind!', 624),
 ('Straight flush!', 40)]

In [4]:
df = pd.DataFrame(hand_counts, columns=["Hand_type", "Hand_count"])
df["Probabilities %"] = df.apply(lambda x: x.Hand_count/len(hands)*100, axis=1)
df

,Hand_type,Hand_count,Probabilities %
0,High card,1302540,50.117739
1,1 pair,1098240,42.256903
2,2 pair,123552,4.753902
3,3 of a kind,54912,2.112845
4,Straight,10200,0.392465
5,Flush,5108,0.196540
6,Full house,3744,0.144058
7,4 of a kind!,624,0.024010
8,Straight flush!,40,0.001539


- Shows how common a *High Card* handrank would be dealt ≈ 50%.
- On the other hand, we see how uncommon a *Straight Flush* is ≈ 0.0015%.
- Probability of a *Straight Flush = 0.001539* is not only unintuitive, it's ugly.
- Can we do better?

In [5]:
from fractions import Fraction # so that we dont have absurdly small percentages e.g. (1/10000)
print("Probabaility of a Straight Flush occuring:")
print("Ugly: ", 40/len(hand_ranks))
print("Pretty: ", Fraction(40, len(hand_ranks)))

Probabaility of a Straight Flush occuring:
Ugly:  1.5390771693292702e-05
Pretty:  1/64974


### Fractions help explain the probability easier.  $\frac{1}{64974}$
This is telling us the proabability of a *Straight Flush* occuring is $1$ out of $64974$
- In other words, we *expect* to see a straight flush once every $64974$ hands. 
- Now that we know the probability of seeing the least common hand, we can set up for simulations.  
    + Given we expect to see a straight flush once approximately every $65k$ hands, let's set the number of simulations so that we expect to see 10 straight flushes, ≈$650K$ hands. 
    
### Lets see who wins!

In [6]:
def winning_hand_distribution(n=700, players=10, cards=5):
    '''
    sample `n` random hands to generate a distribution of 
    winning hand types for # of `players`
    '''
    winners = [0]*10
    for i in range(n):
        hands = deal(players, cards)
        if cards == 7:
            best_combos = [best_hand(h) for h in hands]
            hand_type = hand_rank(poker(best_combos)[0])[0]
        else:
            hand_type = hand_rank(poker(hands)[0])[0]
        winners[hand_type] += 1
    winners.pop(8)# remove index 8
    return winners

In [7]:
five_card_stud_winners = winning_hand_distribution(65_000, players=10, cards=5)
five_card_stud_winners

[108, 29681, 18894, 11577, 2439, 1199, 925, 162, 15]

In [8]:
seven_card_stud_winners = winning_hand_distribution(65_000, players=7, cards=7)
seven_card_stud_winners

[2, 2443, 18049, 9611, 13003, 10436, 10534, 773, 149]

In [9]:
total = 65_000
df["5card_stud_wins"] = five_card_stud_winners
df["5card_prob %"] = df.apply(lambda x: (x["5card_stud_wins"]/total)*100, axis=1)
df["7card_stud_wins"] = seven_card_stud_winners
df["7card_prob %"] = df.apply(lambda x: (x["7card_stud_wins"]/total)*100, axis=1)
df

,Hand_type,Hand_count,Probabilities %,5card_stud_wins,5card_prob %,7card_stud_wins,7card_prob %
0,High card,1302540,50.117739,108,0.166154,2,0.003077
1,1 pair,1098240,42.256903,29681,45.663077,2443,3.758462
2,2 pair,123552,4.753902,18894,29.067692,18049,27.767692
3,3 of a kind,54912,2.112845,11577,17.810769,9611,14.786154
4,Straight,10200,0.392465,2439,3.752308,13003,20.004615
5,Flush,5108,0.196540,1199,1.844615,10436,16.055385
6,Full house,3744,0.144058,925,1.423077,10534,16.206154
7,4 of a kind!,624,0.024010,162,0.249231,773,1.189231
8,Straight flush!,40,0.001539,15,0.023077,149,0.229231


To be continued...

____